# First and Second Derivative of Function

In [ ]:
using ITensors
using Plots
using HDF5

Define function of interest

In [ ]:
N = 8; # Number of sites
s = siteinds("Qubit", N); # Indices for all sites
xs = range(0, 2pi*(1-(1/2^N)), length=2^N); # Sampling grid points
step_size = 2pi/(2^N);

f(x) = cos(x).^2;
ys = f.(xs);

plot(xs, ys) # Plotting the function

Encode function as MPS

In [ ]:
cutoff = 1E-16;
χ = 30;

ψ = MPS(ys, s, cutoff=cutoff, maxdim=χ);

orthogonalize!(ψ,1)

@show linkdims(ψ);

Define MPO for first and second derivatives, using central finite difference up to second order

In [ ]:
# Left shift:

left_ls = zeros(2, 2, 2) 
middle_ls = zeros(2, 2, 2, 2) 
right_ls = zeros(2, 2, 2) 

left_ls[1, 2, 2] = 1
left_ls[2, 1, 1] = 1
middle_ls[1, 1, 1, 1] = 1
middle_ls[1, 2, 2, 1] = 1
middle_ls[2, 2, 1, 1] = 1
middle_ls[2, 1, 2, 2] = 1
right_ls[1, 1, 1] = 1
right_ls[1, 2, 2] = 1
right_ls[2, 1, 2] = 1
right_ls[2, 2, 1] = 1

H_ls = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_ls[i] = ITensor(left_ls, s[i], s[i]', α[i]);
    elseif i == N # last site
        H_ls[i] = ITensor(right_ls, α[i-1], s[i], s[i]');
    else # middle site
        H_ls[i] = ITensor(middle_ls, α[i-1], s[i], s[i]', α[i]);
    end
end

orthogonalize!(H_ls,1);

# Right shift:

left_rs = zeros(2, 2, 2) 
middle_rs = zeros(2, 2, 2, 2) 
right_rs = zeros(2, 2, 2) 

left_rs[1, 2, 1] = 1
left_rs[2, 1, 2] = 1
middle_rs[1, 1, 1, 1] = 1
middle_rs[1, 2, 2, 1] = 1
middle_rs[2, 1, 2, 1] = 1
middle_rs[2, 2, 1, 2] = 1
right_rs[1, 1, 1] = 1
right_rs[1, 2, 2] = 1
right_rs[2, 1, 2] = 1
right_rs[2, 2, 1] = 1

H_rs = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_rs[i] = ITensor(left_rs, s[i], s[i]', α[i]);
    elseif i == N # last site
        H_rs[i] = ITensor(right_rs, α[i-1], s[i], s[i]');
    else # middle site
        H_rs[i] = ITensor(middle_rs, α[i-1], s[i], s[i]', α[i]);
    end
end

orthogonalize!(H_rs,1);

# Identity:

left_id = zeros(2, 2, 1) 
middle_id = zeros(1, 2, 2, 1) 
right_id = zeros(1, 2, 2) 

left_id[1, 1, 1] = 1
left_id[2, 2, 1] = 1
middle_id[1, 1, 1, 1] = 1
middle_id[1, 2, 2, 1] = 1
right_id[1, 1, 1] = 1
right_id[1, 2, 2] = 1

H_id = MPO(N);

α = [Index(1) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_id[i] = ITensor(left_id, s[i], s[i]', α[i]);
    elseif i == N # last site
        H_id[i] = ITensor(right_id, α[i-1], s[i], s[i]');
    else # middle site
        H_id[i] = ITensor(middle_id, α[i-1], s[i], s[i]', α[i]);
    end
end

orthogonalize!(H_id,1);

# Define derivatives
#H_dudx = 0.5*(H_ls - H_rs)/step_size; # First derivative
#H_d2udx2 = (H_rs + H_ls - 2*H_id)/(step_size^2); # Second derivative


In [ ]:
#hh=deepcopy(H_rs);
#C=MPO(N);
#C = prime(hh)*H_rs*H_rs*H_rs
H_rs4=apply(H_rs,H_rs,H_rs,H_rs);
H_ls4=apply(H_ls,H_ls,H_ls,H_ls);
H_rs3=apply(H_rs,H_rs,H_rs);
H_ls3=apply(H_ls,H_ls,H_ls);
H_rs2=apply(H_rs,H_rs);
H_ls2=apply(H_ls,H_ls);

H_dudx= (1/280*H_rs4-4/105*H_rs3+1/5*H_rs2-4/5*H_rs+4/5*H_ls-1/5*H_ls2+4/105*H_ls3-1/280*H_ls4)/step_size;
H_d2udx2 = (-1/560*H_rs4+8/315*H_rs3-1/5*H_rs2+8/5*H_rs-205/72*H_id+8/5*H_ls-1/5*H_ls2+8/315*H_ls3-1/560*H_ls4)/(step_size^2);

In [ ]:

f(x) = -sin(2*x);
yd = f.(xs);
f(x) = -2 * cos(2x);
ydd = f.(xs);

plot(xs, yd) 
plot!(xs, ydd) 

Apply derivatives to MPS encoding function

In [ ]:
Deriv1 = apply(H_dudx,ψ);
Deriv2 = apply(H_d2udx2,ψ);

@show linkdims(Deriv1);
@show linkdims(Deriv2);

In [ ]:
@show linkdims(ψ);
@show linkdims(H_dudx);

Contract the resulting MPS and plot the results

In [ ]:
# Recovering the original big tensors
Big_ψ1 = contract(Deriv1); 
Big_ψ2 = contract(Deriv2); 

# Changing the format from ITensor to Julia array for plotting
J_recon1 = Array(Big_ψ1, s) 
J_recon2 = Array(Big_ψ2, s) 

# Reshaping from N-dimentional tensor to regular array
ys_recon1 = reshape(J_recon1,2^N,1) 
ys_recon2 = reshape(J_recon2,2^N,1) 

# Plotting the reconstructed derivatives of the function
plot(xs, ys_recon1,label="Frist df",linewidth=2) 
plot!(xs, ys_recon2,label="Second df",linewidth=2) 
plot!(xs, yd,label="real f1",linewidth=6,linestyle=:dash) 
plot!(xs, ydd,label="real f2",linewidth=6,linestyle=:dash) 

Exercise finished!!!

# Problem2

In [ ]:
N = 8; # Number of sites
s_x = siteinds("Qubit", N); # Indices for all sites
xb = range(0, (1-(1/2^N)), length=2^N); # Sampling grid points

In [ ]:

step_size = 1/(2^N);
Re=1000;
dt=0.001;
final_t=0.24;
tb = range(0, final_t, step=dt);
f(x) = exp(-8*pi*(x-0.5).^2);
yb = f.(xb);

plot(xb, yb)

In [ ]:
utoff = 1E-30;
χ = 8;

ϕ = MPS(yb, s_x, cutoff=cutoff, maxdim=χ);

orthogonalize!(ϕ,1)

@show linkdims(ϕ);

In [ ]:
# Left shift:

left_ls = zeros(2, 2, 2) 
middle_ls = zeros(2, 2, 2, 2) 
right_ls = zeros(2, 2, 2) 

left_ls[1, 2, 2] = 1
left_ls[2, 1, 1] = 1
middle_ls[1, 1, 1, 1] = 1
middle_ls[1, 2, 2, 1] = 1
middle_ls[2, 2, 1, 1] = 1
middle_ls[2, 1, 2, 2] = 1
right_ls[1, 1, 1] = 1
right_ls[1, 2, 2] = 1
right_ls[2, 1, 2] = 1
right_ls[2, 2, 1] = 1

H_ls = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_ls[i] = ITensor(left_ls, s_x[i], s_x[i]', α[i]);
    elseif i == N # last site
        H_ls[i] = ITensor(right_ls, α[i-1], s_x[i], s_x[i]');
    else # middle site
        H_ls[i] = ITensor(middle_ls, α[i-1], s_x[i], s_x[i]', α[i]);
    end
end

orthogonalize!(H_ls,1);

# Right shift:

left_rs = zeros(2, 2, 2) 
middle_rs = zeros(2, 2, 2, 2) 
right_rs = zeros(2, 2, 2) 

left_rs[1, 2, 1] = 1
left_rs[2, 1, 2] = 1
middle_rs[1, 1, 1, 1] = 1
middle_rs[1, 2, 2, 1] = 1
middle_rs[2, 1, 2, 1] = 1
middle_rs[2, 2, 1, 2] = 1
right_rs[1, 1, 1] = 1
right_rs[1, 2, 2] = 1
right_rs[2, 1, 2] = 1
right_rs[2, 2, 1] = 1

H_rs = MPO(N);

α = [Index(2) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_rs[i] = ITensor(left_rs, s_x[i], s_x[i]', α[i]);
    elseif i == N # last site
        H_rs[i] = ITensor(right_rs, α[i-1], s_x[i], s_x[i]');
    else # middle site
        H_rs[i] = ITensor(middle_rs, α[i-1], s_x[i], s_x[i]', α[i]);
    end
end

orthogonalize!(H_rs,1);

# Identity:

left_id = zeros(2, 2, 1) 
middle_id = zeros(1, 2, 2, 1) 
right_id = zeros(1, 2, 2) 

left_id[1, 1, 1] = 1
left_id[2, 2, 1] = 1
middle_id[1, 1, 1, 1] = 1
middle_id[1, 2, 2, 1] = 1
right_id[1, 1, 1] = 1
right_id[1, 2, 2] = 1

H_id = MPO(N);

α = [Index(1) for i in 1:(N-1)]; # bond indices

for i = 1:N
    if i == 1 # first site
        H_id[i] = ITensor(left_id, s_x[i], s_x[i]', α[i]);
    elseif i == N # last site
        H_id[i] = ITensor(right_id, α[i-1], s_x[i], s_x[i]');
    else # middle site
        H_id[i] = ITensor(middle_id, α[i-1], s_x[i], s_x[i]', α[i]);
    end
end

orthogonalize!(H_id,1);

# Define derivatives
#H_dudx = 0.5*(H_ls - H_rs)/step_size; # First derivative
#H_d2udx2 = (H_rs + H_ls - 2*H_id)/(step_size^2); # Second derivative


In [ ]:
#hh=deepcopy(H_rs);
#C=MPO(N);
#C = prime(hh)*H_rs*H_rs*H_rs
H_rs4=apply(H_rs,H_rs,H_rs,H_rs);
H_ls4=apply(H_ls,H_ls,H_ls,H_ls);
H_rs3=apply(H_rs,H_rs,H_rs);
H_ls3=apply(H_ls,H_ls,H_ls);
H_rs2=apply(H_rs,H_rs);
H_ls2=apply(H_ls,H_ls);

H_dudx= (1/280*H_rs4-4/105*H_rs3+1/5*H_rs2-4/5*H_rs+4/5*H_ls-1/5*H_ls2+4/105*H_ls3-1/280*H_ls4)/step_size;
H_d2udx2 = (-1/560*H_rs4+8/315*H_rs3-1/5*H_rs2+8/5*H_rs-205/72*H_id+8/5*H_ls-1/5*H_ls2+8/315*H_ls3-1/560*H_ls4)/(step_size^2);

In [ ]:
f(x) = -8*pi*exp(-2*pi*(1 - 2*x).^2) *(-1 + 2*x)
yd = f.(xb);
f(x) = 16*pi*exp(-2*pi*(1-2*x).^2)*(4*pi*(1-2*x).^2-1);
ydd = f.(xb);

In [ ]:
Deriv_x1 = apply(H_dudx,ϕ);
Deriv_x2 = apply(H_d2udx2,ϕ);

In [ ]:

Big_ϕ1 = contract(Deriv_x1); 
Big_ϕ2 = contract(Deriv_x2); 

# Changing the format from ITensor to Julia array for plotting
J_recon1 = Array(Big_ϕ1, s_x) 
J_recon2 = Array(Big_ϕ2, s_x) 

# Reshaping from N-dimentional tensor to regular array
ys_recon1 = reshape(J_recon1,2^N,1) 
ys_recon2 = reshape(J_recon2,2^N,1) 

# Plotting the reconstructed derivatives of the function
plot(xb, ys_recon1,label="Frist df",linewidth=2) 
plot!(xb, ys_recon2,label="Second df",linewidth=2) 
plot!(xb, yd,label="real f1",linewidth=6,linestyle=:dash) 
plot!(xb, ydd,label="real f2",linewidth=6,linestyle=:dash) 
#plot!(xb, yb)

In [ ]:
#create hadamard_product_MPO
had = MPO(N);
had .= convert(MPO, ϕ)
for i = 1:N
    had[i] *= δ(s_x[i], s_x[i]', s_x[i]'')
end

had = replaceprime(had, 2 => 0)

#ψ3_Had = apply(had, Deriv_x1)

#@show linkdims(ψ3_Had);

In [ ]:
for χ = [4,6,7];
    cutoff = 1E-30;

    Re=1000;
    ϕ = MPS(yb, s_x, cutoff=cutoff, maxdim=χ);

    orthogonalize!(ϕ,1)
    t=0;
    runs=0;
    #u_xt = []
    saved_times = [0.0, 0.06, 0.12, 0.18, 0.24]
    saved_steps = saved_times./dt;

    #P=plot(xb, yb,label= "t=0")
    #println("00")
    P=plot()
    while t <= final_t+0.01
        Deriv1 = apply(H_dudx,ϕ,maxdim=χ);
        ϕ_Had = apply(had, Deriv1,maxdim=χ);
        Deriv2 = apply(H_d2udx2,ϕ,maxdim=χ);

        ϕ_next = +(ϕ,(dt/Re)*Deriv2,(-dt)*ϕ_Had  ; maxdim=χ);
        
        if runs in saved_steps
            Big_ϕ = contract(ϕ_next); 
            J_recon = Array(Big_ϕ, s_x) 
            y_recon = reshape(J_recon,2^N,1) 
            plot!(xb, y_recon, label="t = $(round(t, digits=2))", lw=2)
            title!("χ = $χ")
            #println("t=$t")
            #plot!(P,pl)
        end
        runs+=1;
        ϕ = ϕ_next;
        t=t+dt;
    end
    display(P)
    
end


In [ ]:
using Plots
χ =8
ϕ = MPS(yb, s_x, cutoff=cutoff, maxdim=χ);

orthogonalize!(ϕ,1)
t=0
#dt =1e-4;
tmax=0.5;
nmax=round(tmax/dt);
anim = Animation()
for n=1:nmax
    Deriv1 = apply(H_dudx,ϕ);
    ϕ_Had = apply(had, Deriv1);
    Deriv2 = apply(H_d2udx2,ϕ);

    ϕ_next = +(ϕ,(dt/Re)*Deriv2,(-dt)*ϕ_Had  ; maxdim=χ);
    
    
    Big_ϕ = contract(ϕ_next); 
    J_recon = Array(Big_ϕ, s_x) 
    y_recon = reshape(J_recon,2^N,1) 
    p=plot(xb, y_recon, legend= false ,title="t = $(round(t, digits=2))", lw=2)
    #println("t=$t")
    frame(anim,p)
    ϕ = ϕ_next;
    t=t+dt;
end
gif(anim, "burger_equation.gif", fps=100)

In [ ]:
mp4(anim, "burger_equation.mp4", fps=100)


# problem4

In [ ]:
N = 30 # System size
J1 = 1; # Hopping parameters
J2 = 0.5;
h=2;
#PBC = 0; # Periodic boundary conditions

# Define N S=1/2 spin indices
sites = siteinds("S=1/2",N);

In [ ]:
function Hamiltonian(N,sites,J1,J2,h)

    # Initiate construction
    ops = OpSum()

    # Hopping terms
    for j=1:N-1
        ops += J1,"Sz",j,"Sz",j+1
        ops += J1,"Sz",j,"Sz",j+1    
    end
    for j=1:N-2
        ops += J2,"Sz",j,"Sz",j+2
        ops += J2,"Sz",j,"Sz",j+2    
    end
    for j=1:N
        ops += -h,"Sx",j
    end

    # PBC if required
   

    H = MPO(ops,sites);
    
    return H;
    
end;

In [ ]:
function InitialState(sites,N)
    
    state = [isodd(n) ? "Up" : "Dn" for n=1:N];
    ψi = MPS(sites,state);
    
    return ψi;
end

In [ ]:
function GroundState(H,ψi)

    # Do many sweeps of DMRG, gradually increasing the maximum MPS bond dimension
    nsweeps = 50;
    maxdim = 100;
    cutoff = 1E-10;

    # Run the DMRG algorithm for ground state   
    energy0,ψ0 = dmrg(H,ψi; nsweeps, maxdim, cutoff, outputlevel=1);

    @show energy0

    # Orthonormalize state. Leave site 1 as orthogonalization center
    orthogonalize!(ψ0,1)

    return energy0, ψ0;
    
end

In [ ]:
# Define spin Hamiltonian
H = Hamiltonian(N,sites,J1,J2,h);
ψi = InitialState(sites,N);
# Calculate ground state
E0,ψ0 = GroundState(H,ψi);

In [ ]:
E0 = inner(ψ0',H,ψ0);
@show E0;

In [ ]:
H2 = inner(H,ψ0,H,ψ0)
var = H2-E^2
@show var

In [ ]:
# Calculate ground state expectation values
Norm = real(norm(ψ0)); # Norm of the state 
Sx = real(expect(ψ0, "Sx"))/Norm^2; # Magnetization along x
Sz = real(expect(ψ0, "Sz"))/Norm^2; # Magnetization along z
SxSx = correlation_matrix(ψ0,"Sx","Sx")/Norm^2; # XX Correlations
SzSz = correlation_matrix(ψ0,"Sz","Sz")/Norm^2; # ZZ Correlations

In [ ]:
plot(Sx, xlabel = "j", ylabel = "Sx(j)")

In [ ]:
heatmap(SzSz, xlabel = "j", ylabel = "k")

In [ ]:
mutable struct DmrgObserver <: AbstractObserver
    energy_tol::Float64
    last_energy::Float64
 
    DmrgObserver(energy_tol=0.0) = new(energy_tol,1000.0)
 end

In [ ]:
function ITensors.checkdone!(o::DmrgObserver;kwargs...)
    sw = kwargs[:sweep]
    energy = kwargs[:energy]
  
    # Stop DMRG if current sweep sequence converged
    if (abs(energy-o.last_energy)/abs(energy) < o.energy_tol)  
      println("Stopping DMRG after sweep $sw") 
      return true
    end
      
    # Otherwise, update last_energy and keep going
    o.last_energy = energy
    return false
  end

In [ ]:
function ExcitedState(H,ψi,ψ0)

    # Do many sweeps of DMRG, gradually increasing the maximum MPS bond dimension
    nsweeps = 50;
    #maxdim = [10 20 30 50 80 100 120]
    maxdim= 200
    cutoff = [1E-10]
    #noise = [1E-6]; # Could also use setnoise!
    noise = [1E+44]
    obs = DmrgObserver(1E-15) # Convergence criterium

    # Run the DMRG algorithm for ground state   
    energy1,ψ1 = dmrg(H,[ψ0],ψi; nsweeps, maxdim, cutoff, noise, observer=obs, outputlevel=1)
    
    @show energy1

    # Orthonormalize state. Leave site 1 as orthogonalization center
    orthogonalize!(ψ1,1)

    return energy1, ψ1;
    
end

In [ ]:
ψi = InitialState(sites,N);

In [ ]:
E1,ψ1 = ExcitedState(H,ψi,ψ0);
Gap = E1 - E0;

@show Gap;

In [ ]:
Ein = inner(ψ0,ψ1);
@show Ein;

In [ ]:
E1 = inner(ψ1',H,ψ1);
@show E1;

In [ ]:
H2 = inner(H,ψ1,H,ψ1)
var = H2-E1^2
@show var